# Part 1: Cleaning and tokenizing the data. df_big refers to the whole 995K FakeNewsCorpus dataset and df refers to the sample. 


In [ ]:
#importing packages
import re
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from cleantext import clean
import nltk
from nltk.corpus import stopwords
from functools import reduce
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import FeatureHasher


## Task 1

In [ ]:
df= pd.read_csv('news_sample.csv', index_col=0)

In [ ]:
#Cleaning data using the cleantext library
def cleantext_clean(data):
    #As clean did not have arguments for date we wrote our own using regular expressions 
    date_formats = [
        r'\d{2,4}/\d{1,2}/\d{2,4}(.*)',   # e.g 12/31/2021 or 12/31/21
        r'\d{2,4}-\d{1,2}-\d{2,4}(.*)',   # e.g 12-31-2021 or 12-31-21
        ]

    for date_format in date_formats:
        data = re.sub(date_format, '<DATE>', str(data))

    data = clean(data, lower=True, normalize_whitespace=True, no_urls=True, no_emails = True, no_numbers= True,
    replace_with_url="<URL>",
    replace_with_email="<EMAIL>",
    replace_with_number="<NUM>",)
    return data
    
for column in df.columns:
    df[column] = df[column].apply(cleantext_clean)



In [ ]:
#Calculating the size of the vocabulary before processing
def word_list(data): 
    word_list = []
    for column in data.columns: 
        words = data[column].tolist()
        words = [word for segments in words for word in str(segments).split()]
        word_list = word_list + words
    return word_list
vc_size_before_processing = len(set(word_list(df)))

In [ ]:
#Removing stopwords and stemming
def remove_stopwords(data):
    stop_words = set(stopwords.words('english'))
    words = str(data).split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def stem_input(data):
    ps = PorterStemmer()
    words = str(data).split()
    return reduce(lambda x, y: x + ' ' + ps.stem(y), words, '')

for column in df.columns:
    df[column] = df[column].apply(remove_stopwords)


In [ ]:
#Calculating the size of the vocabulary after removing stopwords and stemming
vc_size_no_stopwords = len(set(word_list(df)))
print(vc_size_before_processing)
print(vc_size_no_stopwords)

reduction_rate_stopwords = ((vc_size_before_processing-vc_size_no_stopwords)/ vc_size_before_processing) * 100
print(reduction_rate_stopwords, '%')

for column in df.columns:
    df[column] = df[column].apply(stem_input)

vc_size_stemming = len(set(word_list(df)))
reduction_rate_stemming = ((vc_size_before_processing-vc_size_stemming) / vc_size_before_processing) * 100
print(reduction_rate_stemming, '%')
print(vc_size_stemming)

## Task 2, 3. df_big_cleaned refers to the cleaned big dataset, which we did locally and saved to a csv-file. 

In [ ]:
df_big= pd.read_csv('995,000_rows.csv', index_col=0)
df_big_cleaned = pd.read_csv('cleaned_dataset.csv') 

In [ ]:
#Counting how many NaN-values are in the author column
nan_count = df_big['authors'].isna().sum()

(nan_count/ len(df_big['authors'])) * 100

df1 = df_big.copy()
df1['type'] = df1['type'].replace(['bias','conspiracy','fake','hate','junksci','satire'],'fake')
df1['type'] = df1['type'].replace(['political','reliable','clickbait'],'reliable')

df_fake = df1[df1['type'] == 'fake']
df_reliable = df1[df1['type'] == 'reliable']

print ("fake author nan %: ", ((df_fake['authors'].isna().sum()/ len(df_fake['authors']) * 100)))
print ("reliable author nan %: ", ((df_reliable['authors'].isna().sum()/ len(df_reliable['authors']) * 100)))

In [ ]:
#Creates a pandas Series of the frequency of every word in the data to calculate how often each word appears
def word_frequency(data_frame):
    word_freq = {}
    for column in data_frame.columns:
        text = data_frame[column]
        all_text = ' '.join(str(item) for item in text)
        words = all_text.split()
        for word in words:
            if (word != "<num>") and (word != "<num>,") and (word != "<num>.") and (word != "<num>:") and (word != "<url>") and (word != "<email>") and (word != "nan") and (word != "<date>") and (word != ",")and (word != ".") and (word != "-") and (word != "/") and (word != "['']"):
                word_freq[word] = word_freq.get(word, 0) + 1
    
    sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
    return sorted_word_freq

frequencies_before = word_frequency(df_big)
len(frequencies_before)

In [ ]:
#100 most frequent words before cleaning
hundred_most_frequent_before = frequencies_before[:100]

words, freqs = zip(*hundred_most_frequent_before)

plt.figure(figsize=(15, 6))
plt.bar(words, freqs)
plt.xlabel('word')
plt.ylabel('frequency')
plt.title('Frequency of the 100 most used words in the dataset before cleaning')
plt.xticks(rotation=90)
plt.tight_layout() 
plt.show()

In [ ]:
#100 most frequent words after cleaning 
frequencies_after = word_frequency(df_big_cleaned)
hundred_most_frequent_after = frequencies_after[:100]

words, freqs = zip(*hundred_most_frequent_after)
plt.figure(figsize=(15, 6))
plt.bar(words, freqs)
plt.xlabel('word')
plt.ylabel('frequency')
plt.title('Frequency of the 100 most used words in the dataset after cleaning')
plt.xticks(rotation=90)
plt.tight_layout() 
plt.show()

In [ ]:
#Grouping articles by domain and counting the number of articles for each domain
domain_counts = df_big['domain'].value_counts()
domain_counts_df = pd.DataFrame(domain_counts.reset_index())
domain_counts_df.columns = ['Domain', 'Count']

#Creating a boxplot
plt.figure(figsize=(10, 6))
domain_counts_df.boxplot(column='Count', vert=False)
plt.title('Domain Distribution Boxplot')
plt.xlabel('Number of Articles')
plt.yticks([])
plt.grid(True)
plt.show()

domain_counts_df.describe()

In [ ]:
#Calculate the distribution of domains 
domain_distribution = df_big_cleaned['domain'].value_counts()
percentage_distribution = domain_distribution / domain_distribution.sum() * 100
print(percentage_distribution)

type_distribution = df_big_cleaned['type'].value_counts()
percentage_distribution = type_distribution / type_distribution.sum() * 100
print(percentage_distribution)

## Task 4

In [ ]:
#Splitting data into training, validation and test data
x=df_big_cleaned.drop(columns=['type'])
y=df_big_cleaned['type']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42)
x_validation, x_test, y_validation, y_test = train_test_split(x_test, y_test, test_size=0.5,random_state=42)